In [1]:
INIT_LEARNING_RATE = 0.0001

In [2]:
from tensorflow.keras.utils import Sequence
from tensorflow.keras.optimizers import Adam

In [3]:
opt = Adam(learning_rate=INIT_LEARNING_RATE, amsgrad=True)

In [4]:
def depth_loss(y_true, y_pred):
    w1, w2, w3 = 1.0, 3.0, 0.1
    
    l_depth = K.mean(K.abs(y_pred - y_true), axis=-1)
    
    dy_true, dx_true = tf.image.image_gradients(y_true)
    dy_pred, dx_pred = tf.image.image_gradients(y_pred)
    l_edges = K.mean(K.abs(dy_pred - dy_true) + K.abs(dx_pred - dx_true), axis=-1)
    print(l_edges)
    
    l_ssim = K.clip((1 - tf.image.ssim(y_true, y_pred, 1.0)) * 0.5, 0, 1)
    
    return (w1 * l_ssim) + (w2 * K.mean(l_edges)) + (w3 * K.mean(l_depth))

In [5]:
def depth_acc(y_true, y_pred):
    return K.mean(K.equal(K.round(y_true), K.round(y_pred)))

In [6]:
class DataGenerator(Sequence):
    def __init__(self, list_IDs, labels, batch_size=16, dim=(128, 128), n_channels=3, shuffle=True, pred=False):
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.pred = pred
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))
    
    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        if self.pred:
            X = self.__data_generation(list_IDs_temp)
            return X
        X, y = self.__data_generation(list_IDs_temp)
        return X, y
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, self.dim[0], self.dim[1], self.n_channels))
        
        if not self.pred:
            y = np.empty((self.batch_size, self.dim[0], self.dim[1], 1))
        
            for i, ID in enumerate(list_IDs_temp):
                res = random.choice([True, False])
                X[i,] = preprocess_image(ID, res)
                y[i,] = preprocess_depth_map(self.labels[ID], res)
            return X, y
        else:
            for i, ID in enumerate(list_IDs_temp):
                res = random.choice([True, False])
                X[i,] = preprocess_image(ID, res)
            return X